In [1]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()
using LinearAlgebra
using ForwardDiff
using RobotZoo
using RobotDynamics
using Ipopt
using MathOptInterface
using TrajOptPlots
const MOI = MathOptInterface
using Random
include("quadratic_cost.jl")
include("planar_quadruped.jl")
include("sparseblocks.jl")
include("utils.jl")

  Activating environment at `~/Documents/16745/homework4-Atom990 (copy)/Project.toml`


WebIO._IJuliaInit()

slerp (generic function with 1 method)

In [ ]:
model = PlanarQuadruped()
x, u = rand(model)  # generate some random states and controls
dt = 0.1

# evaluate the discrete dynamics using RK4
contact1_dynamics_rk4(model, x, u, dt)
contact2_dynamics_rk4(model, x, u, dt)

# jump maps
jump1_map(x)
jump2_map(x)

# evaluate the discrete dynamics Jacobians
contact1_jacobian(model, x, u, dt)
contact2_jacobian(model, x, u, dt)
contact3_jacobian(model, x, u, dt)

# jump map Jacobian
jump1_jacobian()
jump2_jacobian();

## Setting up the NLP

In [ ]:
"""
    HybridNLP{n,m,L,Q}

Represents a (N)on(L)inear (P)rogram of a trajectory optimization problem,
with a dynamics model of type `L`, a quadratic cost function, horizon `T`, 
and initial and final state `x0`, `xf`.

The kth state and control can be extracted from the concatenated state vector `Z` using
`Z[nlp.xinds[k]]`, and `Z[nlp.uinds[k]]`.

# Constructor
    HybridNLP(model, obj, tf, N, M, x0, xf, [integration])

# Basic Methods
    Base.size(nlp)    # returns (n,m,T)
    num_ineq(nlp)     # number of inequality constraints
    num_eq(nlp)       # number of equality constraints
    num_primals(nlp)  # number of primal variables
    num_duals(nlp)    # total number of dual variables
    packZ(nlp, X, U)  # Stacks state `X` and controls `U` into one vector `Z`

# Evaluating the NLP
The NLP supports the following API for evaluating various pieces of the NLP:

    eval_f(nlp, Z)         # evaluate the objective
    grad_f!(nlp, grad, Z)  # gradient of the objective
    eval_c!(nlp, c, Z)     # evaluate the constraints
    jac_c!(nlp, c, Z)      # constraint Jacobian
"""
struct HybridNLP{n,m,L,Q} <: MOI.AbstractNLPEvaluator
    model::L                                 # dynamics model
    obj::Vector{QuadraticCost{n,m,Float64}}  # objective function
    N::Int                                   # number of knot points
    Nmodes::Int                              # number of modes
    init_mode::Int                           # the mode ID of the initial state
    t_trans::Float64                         # the staet time of mode 3 (sec)
    tf::Float64                              # total time (sec)
    x0::MVector{n,Float64}                   # initial condition
    xf::MVector{n,Float64}                   # final condition
    times::Vector{Float64}                   # vector of times
    modes::Vector{Int}                       # mode ID
    xinds::Vector{SVector{n,Int}}            # Z[xinds[k]] gives states for time step k
    uinds::Vector{SVector{m,Int}}            # Z[uinds[k]] gives controls for time step k
    cinds::Vector{UnitRange{Int}}            # indices for each of the constraints
    lb::Vector{Float64}                      # lower bounds on the constraints
    ub::Vector{Float64}                      # upper bounds on the constraints
    zL::Vector{Float64}                      # lower bounds on the primal variables
    zU::Vector{Float64}                      # upper bounds on the primal variables
    rows::Vector{Int}                        # rows for Jacobian sparsity
    cols::Vector{Int}                        # columns for Jacobian sparsity
    use_sparse_jacobian::Bool
    blocks::BlockViews
    function HybridNLP(model, obj::Vector{<:QuadraticCost{n,m}}, init_mode::Integer,
            tf::Real, N::Integer, x0::AbstractVector, xf::AbstractVector,
            integration::Type{<:QuadratureRule}=RK4; use_sparse_jacobian::Bool=false
        ) where {n,m}
        # Create indices
        xinds = [SVector{n}((k-1)*(n+m) .+ (1:n)) for k = 1:N]
        uinds = [SVector{m}((k-1)*(n+m) .+ (n+1:n+m)) for k = 1:N-1]
        times = collect(range(0, tf, length=N))
        
        # TODO: Specify the mode sequence
        modes = map(1:N) do k
            (times[k] < t_trans) ? init_mode : 3
        end
        Nmodes = 2

        k_trans = 0
        for k = 1:N-1
            if times[k] < t_trans && times[k+1] >= t_trans
                k_trans = k + 1
                break
            end
        end
        
        # TODO: specify the constraint indices
        c_init_inds = 1:n                                                                               # initial constraint
        c_term_inds = (c_init_inds[end]+1):(c_init_inds[end]+n)                                         # terminal constraint
        c_dyn_inds = (c_term_inds[end]+1):(c_term_inds[end]+(N-1)*n)                                    # dynamics constraints
        c_init_contact_inds = (c_dyn_inds[end]+1):(c_dyn_inds[end]+2*N-1)                                 # contact constraints of the initial mode (2 per time step)
        c_other_contact_inds = (c_init_contact_inds[end]+1):(c_init_contact_inds[end]+2*(N-k_trans)+1)  # contact constraints of another leg (2 per time step)
        c_kin_inds = (c_other_contact_inds[end]+1):(c_other_contact_inds[end]+2*N)                      # kinematic constraints (2 per time step)
        c_col_inds = (c_kin_inds[end]+1):(c_kin_inds[end]+N)                                            # self-collision constraints (1 per time step)
        
        m_nlp = c_col_inds[end]  # total number of constraints
        
        # TODO: specify the bounds on the constraints
        lb = fill(0.0, m_nlp) # lower bounds on the constraints
        ub = fill(0.0, m_nlp) # upper bounds on the constraints

        ub[c_kin_inds] .= model.l1 + model.l2 + model.lb/2
        ub[c_col_inds] .= 2*(model.l1+model.l2) + model.lb

        # TODO: Other initialization
        cinds = [c_init_inds, c_term_inds, c_dyn_inds, c_init_contact_inds, c_other_contact_inds, c_kin_inds, c_col_inds]

        n_nlp = n*N + (N-1)*m
        zL = fill(-Inf, n_nlp)
        zU = fill(+Inf, n_nlp)
        rows = Int[]
        cols = Int[]
        blocks = BlockViews(m_nlp, n_nlp)
        
        new{n,m,typeof(model), integration}(
            model, obj,
            N, Nmodes, init_mode, t_trans, tf, x0, xf, times, modes,
            xinds, uinds, cinds, lb, ub, zL, zU, rows, cols, use_sparse_jacobian, blocks
        )
    end
end
Base.size(nlp::HybridNLP{n,m}) where {n,m} = (n,m,nlp.N)
num_primals(nlp::HybridNLP{n,m}) where {n,m} = n*nlp.N + m*(nlp.N-1)
num_duals(nlp::HybridNLP) = nlp.cinds[end][end]

"""
    packZ(nlp, X, U)

Take a vector state vectors `X` and controls `U` and stack them into a single vector Z.
"""
function packZ(nlp, X, U)
    Z = zeros(num_primals(nlp))
    for k = 1:nlp.N-1
        Z[nlp.xinds[k]] = X[k]
        Z[nlp.uinds[k]] = U[k]
    end
    Z[nlp.xinds[end]] = X[end]
    return Z
end

"""
    unpackZ(nlp, Z)

Take a vector of all the states and controls and return a vector of state vectors `X` and
controls `U`.
"""
function unpackZ(nlp, Z)
    X = [Z[xi] for xi in nlp.xinds]
    U = [Z[ui] for ui in nlp.uinds]
    return X, U
end

# includes the interface to Ipopt
include("moi.jl")

## Costs

In [ ]:
"""
    eval_f(nlp, Z)

Evaluate the objective, returning a scalar.
"""
function eval_f(nlp::HybridNLP, Z)
    J = 0.0
    xi,ui = nlp.xinds, nlp.uinds
    for k = 1:nlp.N-1
        x,u = Z[xi[k]], Z[ui[k]]
        J += stagecost(nlp.obj[k], x, u)
    end
    J += termcost(nlp.obj[end], Z[xi[end]])
    return J
end

"""
    grad_f!(nlp, grad, Z)

Evaluate the gradient of the objective at `Z`, storing the result in `grad`.
"""
function grad_f!(nlp::HybridNLP{n,m}, grad, Z) where {n,m}
    xi,ui = nlp.xinds, nlp.uinds
    obj = nlp.obj
    for k = 1:nlp.N-1
        x,u = Z[xi[k]], Z[ui[k]]
        grad[xi[k]] = obj[k].Q*x + obj[k].q
        grad[ui[k]] = obj[k].R*u + obj[k].r
    end
    grad[xi[end]] = obj[end].Q*Z[xi[end]] + obj[end].q
    return nothing
end

## Reference Trajectory

In [ ]:
"""
    reference_trajectory(model, times)

Return a reference trajectory that translates the walker from an x position of `xinit` to `xterm`,
with a nominal body height of `height` meters.
"""
function reference_trajectory(model::PlanarQuadruped, times, t_trans, xinit, xterm, uinit, init_mode)
    # Some useful variables
    n, m = size(model)
    tf = times[end]
    N = length(times)
    Δx = xterm - xinit
    dt = times[2]

    g, mb, lb, l1, l2 = model.g, model.mb, model.lb, model.l1, model.l2

    # initialization
    k_trans = 0
    for k = 1:N-1
        if times[k] < t_trans && times[k+1] >= t_trans
            k_trans = k + 1
            break
        end
    end

    Xref = zeros(n,N)
    Uref = zeros(m,N)

    Xref[:, 1] .= xinit
    Uref[:, 1] .= uinit

    Xref[:, 2:end] .= xterm

    a = 2 * Δx[7:10] / (t_trans^2)     # acceleration of feet (assume feet have constant acceleration and zero velocity since touching the ground)

    for k = 2:k_trans-1
        Uref[1:4, :] .= uinit[1:4] + a * (k-1)*dt  # v1 and v2
    end

    if init_mode == 1
        Uref[6, :] .= mb*g/2           # F1_y
        Uref[8, k_trans:end] .= mb*g/2    # F2_y
    else
        Uref[6, k_trans:end] .= mb*g/2    # F1_y
        Uref[8, :] .= mb*g/2           # F2_y
    end

    # Convert to a trajectory
    Xref = [SVector{n}(x) for x in eachcol(Xref)]
    Uref = [SVector{m}(u) for u in eachcol(Uref)]
    return Xref, Uref
end

## Problem Definition

In [ ]:
# Dynamics model
model = PlanarQuadruped()
g, mb, lb, l1, l2 = model.g, model.mb, model.lb, model.l1, model.l2

# Discretization
tf = 2.0
dt = 0.05
N = Int(ceil(tf/dt)) + 1
times = range(0,tf, length=N)
t_trans = 1.0

# Initial Conditions
# currently, we assume the initial mode ID is 1
xinit = [-0.4;0.6;3.1415/9; 0.0;-0.5;0.0; 0.0;0.0; -1.0;0.2]
xterm = [-lb/2;sqrt(l1^2+l2^2);0.0; 0.0;0.0;0.0; 0.0;0.0; -lb;0.0]

uinit = [0.0;0.0; 0.0;-0.5; 0.0;mb*g/2; 0.0;mb*g/2]
init_mode = 1

# Reference Trajectory
Xref,Uref = reference_trajectory(model, times, t_trans, xinit, xterm, uinit, init_mode)

# Objective
Random.seed!(1)
Q = Diagonal([10.0;10.0;1.0; 10.0;10.0;1.0; 10.0;10.0; 10.0;10.0])
R = Diagonal(fill(1e-3,8))
Qf = Q

obj = map(1:N-1) do k
    LQRCost(Q,R,Xref[k],Uref[k])
end
push!(obj, LQRCost(Qf, R*0, Xref[N], Uref[1]))

# Define the NLP
nlp = HybridNLP(model, obj, init_mode, tf, N, Xref[1], Xref[end]);

## Constraints

In [ ]:
# TASK: Implement the following methods
#       1. dynamics constraints
#       2. contact constraints of the initial mode (2 per time step)
#       3. contact constraints of another leg (2 per time step)
#       4. kinematic constraints (2 per time step)
#       5. self-collision constraints (1 per time step)

"""
    dynamics_constraint!(nlp, c, Z)

Calculate the dynamics constraints for the hybrid dynamics.
"""
function dynamics_constraint!(nlp::HybridNLP{n,m}, c, Z) where {n,m}
    xi,ui = nlp.xinds, nlp.uinds
    model = nlp.model
    init_mode = nlp.init_mode
    N = nlp.N                      # number of time steps
    dt = nlp.times[2]
    t_trans = nlp.t_trans
    
    # Grab a view of the indices for the dynamics constraints
    d = reshape(view(c, nlp.cinds[3]), n, N-1)
    
    # TODO: calculate the hybrid dynamics constraints
    # TIP: remember to include the jump map when the mode changes!
    k_trans = 0
    for k = 1:N-1
        if times[k] < t_trans && times[k+1] >= t_trans
            k_trans = k + 1
            break
        end
    end

    for k = 1:N-1
        x, u = Z[xi[k]], Z[ui[k]]
        x_next = Z[xi[k+1]]
        if k < k_trans-1  # in mode 1 or 2
            if init_mode == 1
                d[:, k] = contact1_dynamics_rk4(model, x, u, dt) - x_next
            else
                d[:, k] = contact2_dynamics_rk4(model, x, u, dt) - x_next
            end
        elseif k == k_trans-1  # before transition, jump!
            if init_mode == 1
                d[:, k] = jump1_map(contact1_dynamics_rk4(model, x, u, dt)) - x_next
            else
                d[:, k] = jump2_map(contact2_dynamics_rk4(model, x, u, dt)) - x_next
            end
        else  # in mode 3
            d[:, k] = contact3_dynamics_rk4(model, x, u, dt) - x_next
        end
    end

    return vec(d)   # for easy Jacobian checking
end

"""
    contact_init_constraints!(nlp, c, Z)

Calculate the contact constraints of the initial mode for each time step.
"""
function contact_init_constraints!(nlp::HybridNLP{n,m}, c, Z) where {n,m}
    d = view(c, nlp.cinds[4])
    
    # Some useful variables
    xi,ui = nlp.xinds, nlp.uinds
    N = nlp.N                      # number of time steps
    Nmodes = nlp.Nmodes            # number of mode sequences (N ÷ M)
    init_mode = nlp.init_mode
    
    # TODO: Calculate the stance constraints
    for k = 1:N
        x = Z[xi[k]]
        if init_mode == 1
            d[2*k-1] = x[8]
            if k < N
                u = Z[ui[k]]
                d[2*k] = u[2]
            end
        else
            d[2*k-1] = x[10]
            if k < N
                u = Z[ui[k]]
                d[2*k] = u[4]
            end
        end
    end
    
    return d  # for easy Jacobian checking
end

"""
    contact_constraints_other!(nlp, c, Z)
    
Calculate the contact constraints of another leg for each time step after transition
"""
function contact_another_constraints!(nlp::HybridNLP{n,m}, c, Z) where {n,m}
    d = view(c, nlp.cinds[5])
    
    # Some useful variables
    xi,ui = nlp.xinds, nlp.uinds
    N = nlp.N                      # number of time steps
    Nmodes = nlp.Nmodes            # number of mode sequences (N ÷ M)
    init_mode = nlp.init_mode
    t_trans = nlp.t_trans

    k_trans = 0
    for k = 1:N-1
        if times[k] < t_trans && times[k+1] >= t_trans
            k_trans = k + 1
            break
        end
    end

    for k = 1:N-k_trans+1
        x = Z[xi[k]]
        if init_mode == 1
            d[2*k-1] = x[10]
            if k < N-k_trans+1
                u = Z[ui[k]]
                d[2*k] = u[4]
            end
        else
            d[2*k-1] = x[8]
            if k < N-k_trans+1
                u = Z[ui[k]]
                d[2*k] = u[2]
            end
        end
    end

    return d  # for easy Jacobian checking
end

"""
    kinematics_constraints!(nlp, c, Z)

Calculate the kinematics constraints.
"""
function kinematics_constraints!(nlp::HybridNLP{n,m}, c, Z) where {n,m}
    # Create a view for the portion for the length constraints
    d = view(c, nlp.cinds[6])
    
    # Some useful variables
    xi,ui = nlp.xinds, nlp.uinds
    N = nlp.N                      # number of time steps
    Nmodes = nlp.Nmodes            # number of mode sequences (N ÷ M)
    init_mode = nlp.init_mode
    
    for k = 1:N
        x = Z[xi[k]]

        pb = x[1:2]
        p1 = x[7:8]
        p2 = x[9:10]
        
        d[2*k-1] = norm(pb - p1)
        d[2*k] = norm(pb - p2)
    end

    return d   # for easy Jacobian checking
end

"""
    self_collision_constraints!(nlp, c, Z)

Calculate the self-collision constraints.
"""
function self_collision_constraints!(nlp::HybridNLP{n,m}, c, Z) where {n,m}
    # Create a view for the portion for the length constraints
    d = view(c, nlp.cinds[7])
    
    # Some useful variables
    xi,ui = nlp.xinds, nlp.uinds
    N = nlp.N                      # number of time steps
    Nmodes = nlp.Nmodes            # number of mode sequences (N ÷ M)
    init_mode = nlp.init_mode
    
    for k = 1:N
        x = Z[xi[k]]
        
        p1 = x[7:8]
        p2 = x[9:10]
        
        d[k] = norm(p1 - p2)
    end

    return d   # for easy Jacobian checking
end

"""
    eval_c!(nlp, c, Z)

Evaluate all the constraints
"""
function eval_c!(nlp::HybridNLP, c, Z)
    xi = nlp.xinds
    c[nlp.cinds[1]] .= Z[xi[1]] - nlp.x0
    c[nlp.cinds[2]] .= Z[xi[end]] - nlp.xf
    dynamics_constraint!(nlp, c, Z)
    contact_init_constraints!(nlp, c, Z)
    contact_another_constraints!(nlp, c, Z)
    kinematics_constraints!(nlp, c, Z)
    self_collision_constraints!(nlp, c, Z)
end

## Constraint Jacobians

In [ ]:
# TASK: Implement the following methods
#       1. dynamics_jacobian! (9 pts)
#       2. jac_c!             (6 pts)

"""
    dynamics_jacobian!(nlp, jac, Z)

Calculate the Jacobian of the dynamics constraints, storing the result in the matrix `jac`.
"""
function dynamics_jacobian!(nlp::HybridNLP{n,m}, jac, Z) where {n,m}
    # Create a view of the portion of the Jacobian for the dynamics constraints
    D = view(jac, nlp.cinds[3], :)
        
    # Some useful variables
    xi,ui = nlp.xinds, nlp.uinds
    model = nlp.model
    init_mode = nlp.init_mode
    N = nlp.N                      # number of time steps
    dt = nlp.times[2]
    t_trans = nlp.t_trans
    
    # TODO: Calculate the dynamics Jacobians
    ci = 1:n

    k_trans = 0
    for k = 1:N-1
        if times[k] < t_trans && times[k+1] >= t_trans
            k_trans = k + 1
            break
        end
    end

    for k = 1:N-1
        x, u = Z[xi[k]], Z[ui[k]]

        if k < k_trans-1  # in mode 1 or 2
            if init_mode == 1
                D[ci, [xi[k]; ui[k]]] = contact1_jacobian(model, x, u, dt)
            else
                D[ci, [xi[k]; ui[k]]] = contact2_jacobian(model, x, u, dt)
            end
        elseif k == k_trans-1  # before transition, jump!
            if init_mode == 1
                D[ci, [xi[k]; ui[k]]] .= jump1_jacobian() * contact1_jacobian(model, x, u, dt)
            else
                D[ci, [xi[k]; ui[k]]] .= jump2_jacobian() * contact2_jacobian(model, x, u, dt)
            end
        else  # in mode 3
            D[ci, [xi[k]; ui[k]]] = contact3_jacobian(model, x, u, dt)
        end

        D[ci, xi[k+1]] .= -I(n)
        ci = ci .+ n
    end

    return nothing
end

"""
    jac_c!(nlp, jac, Z)

Evaluate the constraint Jacobians.
"""
function jac_c!(nlp::HybridNLP{n,m}, jac, Z) where {n,m}
    xi,ui = nlp.xinds, nlp.uinds
    N = nlp.N
    init_mode = nlp.init_mode
    t_trans = nlp.t_trans

    k_trans = 0
    for k = 1:N-1
        if times[k] < t_trans && times[k+1] >= t_trans
            k_trans = k + 1
            break
        end
    end
    
    # Create views for each portion of the Jacobian
    jac_init = view(jac, nlp.cinds[1], xi[1])
    jac_term = view(jac, nlp.cinds[2], xi[end])
    jac_dynamics = view(jac, nlp.cinds[3], :)
    jac_contact_init = view(jac, nlp.cinds[4], :)
    jac_contact_another = view(jac, nlp.cinds[5], :)
    jac_kinematics = view(jac, nlp.cinds[6], :)
    jac_self_collision = view(jac, nlp.cinds[7], :)
    
    # TODO: Calculate all the Jacobians
    #  TIP: You can write extra functions for the other constraints, or just do them here (they're pretty easy)
    #  TIP: Consider starting with ForwardDiff and then implement analytically (you won't get full points if you don't
    #       implement the Jacobians analytically)
    jac_init .= I(n)
    jac_term .= I(n)
    
    # jac_dynamics
    dynamics_jacobian!(nlp, jac, Z)

    # jac_contact_init
    if init_mode == 1
        for k = 1:N
            jac_contact_init[k, xi[k][8]] = 1
            if k < N
                jac_contact_init[k, ui[k][2]] = 1
            end
        end
    else
        for k = 1:N
            jac_contact_init[k, xi[k][10]] = 1
            if k < N
                jac_contact_init[k, ui[k][4]] = 1
            end
        end
    end

    # jac_contact_another
    if init_mode == 1
        for k = k_trans:N
            i = k - k_trans + 1
            jac_contact_another[i, xi[k][10]] = 1
            if k < N
                jac_contact_another[i, ui[k][4]] = 1
            end
        end
    else
        for k = k_trans:N
            i = k - k_trans + 1
            jac_contact_another[i, xi[k][8]] = 1
            if k < N
                jac_contact_another[i, ui[k][2]] = 1
            end
        end
    end

    # jac_kinematics
    for k = 1:N
        x = Z[xi[k]]
        
        d1 = x[1:2] - x[7:8]
        d2 = x[1:2] - x[9:10]
        
        jac_kinematics[2*k-1, xi[k][1:2]] = d1 / norm(d1)
        jac_kinematics[2*k-1, xi[k][7:8]] = -d1 / norm(d1)
        
        jac_kinematics[2*k, xi[k][1:2]] = d2 / norm(d2)
        jac_kinematics[2*k, xi[k][9:10]] = -d2 / norm(d2)
    end

    # jac_self_collision
    for k = 1:N
        x = Z[xi[k]]

        d12 = x[7:8] - x[9:10]

        jac_self_collision[k, xi[k][7:8]] = d12 / norm(d12)
        jac_self_collision[k, xi[k][9:10]] = -d12 / norm(d12)
    end
    
    return nothing
end

## Solve

In [ ]:
# Initial guess
Random.seed!(1)
Xguess = [x + 0.1*randn(length(x)) for x in Xref]
Uguess = [u + 0.1*randn(length(u)) for u in Uref]
Z0 = packZ(nlp, Xguess, Uguess);
nlp = HybridNLP(model, obj, init_mode, tf, N, Xref[1], Xref[end], use_sparse_jacobian=false);

In [ ]:
# Z_sol, solver = solve(Z0, nlp, c_tol=1e-6, tol=1e-6)
Z_sol, solver = solve(Z0, nlp, c_tol=1e-4, tol=1e-2)